# This notebook uses the UCI banknote dataset
- https://archive.ics.uci.edu/ml/datasets/banknote+authentication

In [ ]:
from sklearn import tree
import graphviz
import os
import numpy as np
from sklearn.model_selection import GridSearchCV
import pandas as pd
from IPython.display import IFrame

## Data set

In [ ]:
data = pd.read_csv('../dataset/data_banknote_authentication.csv')
feats = list(data)[:-1]
data.drop_duplicates(keep='first', inplace=True, ignore_index=True, subset=list(data)[:-1])
neg, pos = [data.loc[data['class'] == arg] for arg in (0, 1)]
total = len(data)
print("negative: {:.0%}, positive: {:.0%}".format(len(neg) / total, len(pos) / total))
data.describe()

## Decision Tree

### Gini impurity measure
- Let $p_{nc}$ denote the density of a class $c$ in a given node $n$
$$G(N_n) = \sum\limits_{c} p_{nc}(1-p_{nc})$$

- Minimizing $G$ encourages the model to make splits such that the data is better separated. When $p_{nc}$ approaches 0, then less instances of $c_i$ exist in $n$ and more instances of $c_j$ exist in $n$. The converse is true as well, that is, when $p_{nc}$ approaches 1, then more instances of $c_j$ exist in $n$ and less instances of $c_i$ exist in $n$. When $p_{nc}$ approaches 0 or 1, then the summation term approaches 0 and the summation approaches 0. Thus, the model is encouraged to choose splits that further segregate the data.

### Cost complexity pruning for binary tree
- Cost complexity pruning is used to regularize the tree and prevent overfitting. Cost complexity pruning can be defined as
$$V = C + \alpha * S$$
where $S$ is a sequence $N, N - 2, N - 4, N - 6, ... N - 2 * N_{int}$ and $N_{int}$ is the number of internal nodes in $N$, parameter $\alpha \geq 0$, and $C$ is a cumulative sum of the sequence $0, C_1, C_1, C_2, ... C_n$. $argmin (V)$ gives the appropriate number of prunes to perform. This can be seen by the fact that as $\alpha$ grows, so does $S$. Therefore, more cost must be accrued to reduce the tree complexity and $argmin(V)$ grows accordingly.
- The prune complexity of a single node can be defined as
$$ P_c = W(t) - W(I_t) - W(J_t) $$
where $t$ is some tree, $I$ and $J$ are children of $t$, and $W$ is the sample weighted impurity of the node. $C$ is a sequence of $P_c$ ordered from least costly to most, and the least costly nodes are pruned first until $argmin(V)$ is surpassed.

In [ ]:
#Create mapping and model
xs, ys = np.asarray(data[list(data)[:-1]]), np.asarray(data[list(data)[-1]])
model = tree.DecisionTreeClassifier()

#CV multiple models, grid search over cost complexity
param_space = {'ccp_alpha': np.linspace(0.001, 0.05, num=25)}
search = GridSearchCV(estimator=model, param_grid=param_space, cv=5)
search.fit(xs, ys)
df = pd.DataFrame(search.cv_results_)[['param_ccp_alpha', 'rank_test_score', 'std_test_score', 'mean_test_score', 'params']]\
    .sort_values(by=['rank_test_score'])
#print cross validation results sorted by rank_test_score
print(df[list(df)[:-1]])

- ccp_alpha of 0.05 was used to see how the tree changes. Obviously, it is too aggresive and causes severe underfitting.
- No visiblity on the trees used in CV. However, we can fit the trees again to the entire dataset during analysis and see how pruning affects the shape.
- Accuracy measurements do not correspond one-to-one between succeeding trees and values in table, but they would be close.

In [ ]:
#This function will be used to draw decision trees
def graph(model, feats, plot_dir, name):
    dot_data = tree.export_graphviz(model, out_file=None, feature_names=feats, 
                                    class_names=['authentic', 'fraudulent'], 
                                    filled=True, rounded=True, special_characters=True, 
                                    leaves_parallel=True)
    graph = graphviz.Source(dot_data)
    out_path = os.path.join(plot_dir, name)
    graph.render(out_path, format='png', cleanup=True)
    return out_path

In [ ]:
overfit_model = tree.DecisionTreeClassifier(**df['params'].iloc[0])
overfit_model.fit(xs, ys)
graph(overfit_model, feats, './graphs', 'overfit')
#Reminder: you can click on the iframe to zoom in
IFrame("./graphs/overfit.png", width=800, height=300)

- This tree overfits the data. There are several buckets that have one sample in them and the tree is too complex. Thus, this tree will not generalize well to unseen data.

In [ ]:
underfit_model = tree.DecisionTreeClassifier(**df['params'].iloc[-1])
underfit_model.fit(xs, ys)
graph(underfit_model, feats, './graphs', 'underfit')
IFrame("./graphs/underfit.png", width=800, height=400)

- Choosing the last hyperparameter shows a tree that underfits because it is lacking in complexity. However, I am amazed to see that we can achieve roughly 88% - 91% with such a simple tree.
- If such a simple tree achieves such high accuracy, then that must speak to the high degree of separability in the data set.

In [ ]:
bestfit_model = tree.DecisionTreeClassifier(**df['params'].iloc[3])
bestfit_model.fit(xs, ys)
graph(bestfit_model, feats, './graphs', 'bestfit_model')
IFrame("./graphs/bestfit_model.png", width=800, height=400)

- The tree at index 3 appears to be far less complex than the one at 0 and has no small buckets.
- This tree has a higher variance than the tree at index 0, and looks to produce roughly 96% - 98% accuracy.